In [5]:
import random
import pickle
import re

wordnet = {}
with open("./wordnet.pickle", "rb") as f:
    wordnet = pickle.load(f)

def get_only_hangul(line):
    # 한글만 남기기
    parseText = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣\s]', '', line)
    return parseText.strip()

def get_synonyms(word):
    synonyms = []
    try:
        for syn in wordnet.get(word, []):
            for s in syn:
                synonyms.append(s)
    except:
        pass
    return synonyms

def synonym_replacement(words, n):
    new_words = words.copy()
    random_word_list = list(set([word for word in words]))
    random.shuffle(random_word_list)
    num_replaced = 0
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        if len(synonyms) >= 1:
            synonym = random.choice(list(synonyms))
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced >= n:
            break

    return new_words

def random_deletion(words, p):
    if len(words) == 1:
        return words

    new_words = []
    for word in words:
        r = random.uniform(0, 1)
        if r > p:
            new_words.append(word)

    if len(new_words) == 0:
        rand_int = random.randint(0, len(words)-1)
        return [words[rand_int]]

    return new_words

def swap_word(new_words):
    if len(new_words) < 2:
        return new_words
    random_idx_1 = random.randint(0, len(new_words)-1)
    random_idx_2 = random_idx_1
    counter = 0
    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words)-1)
        counter += 1
        if counter > 3:
            return new_words
    new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
    return new_words

def random_swap(words, n):
    new_words = words.copy()
    for _ in range(n):
        new_words = swap_word(new_words)
    return new_words

def add_word(new_words):
    synonyms = []
    counter = 0
    while len(synonyms) < 1:
        if len(new_words) >= 1:
            random_word = new_words[random.randint(0, len(new_words)-1)]
            synonyms = get_synonyms(random_word)
            counter += 1
        else:
            # 문장이 빈 경우엔 삽입 불가
            return 
        if counter > 10:
            return
    random_synonym = synonyms[0]
    random_idx = random.randint(0, len(new_words)-1)
    new_words.insert(random_idx, random_synonym)

def random_insertion(words, n):
    new_words = words.copy()
    for _ in range(n):
        add_word(new_words)
    return new_words


def generate_augmented_sentences(sentence, method='sr', num_aug=1):
    # 원본 문장 전처리
    sentence = get_only_hangul(sentence)
    words = sentence.split()
    words = [w for w in words if w]

    augmented_sentences = []

    for _ in range(num_aug):
        if len(words) == 0:
            # 만약 단어가 없다면 그대로 반환
            augmented_sentences.append(sentence)
            continue

        num_words = len(words)

        if method == 'sr':
            # Synonym Replacement
            n_sr = random.randint(1, max(1, num_words))
            new_words = synonym_replacement(words, n_sr)
            augmented_sentences.append(' '.join(new_words))

        elif method == 'ri':
            # Random Insertion
            n_ri = random.randint(1, max(1, num_words))
            new_words = random_insertion(words, n_ri)
            augmented_sentences.append(' '.join(new_words))

        elif method == 'rs':
            # Random Swap
            n_rs = random.randint(1, max(1, num_words))
            new_words = random_swap(words, n_rs)
            augmented_sentences.append(' '.join(new_words))

        elif method == 'rd':
            # Random Deletion
            p_rd = random.uniform(0.1, 0.9)
            new_words = random_deletion(words, p_rd)
            augmented_sentences.append(' '.join(new_words))

        else:
            # 지정된 method가 아닌 경우 원문 반환
            augmented_sentences.append(sentence)

    return augmented_sentences

# 예시 사용
# original_sentence = "이 문장을 증강해 주세요."
# augmented_list = generate_augmented_sentences(original_sentence, method='ri', num_aug=5)
# print(augmented_list)

In [12]:
generate_augmented_sentences("이 슬픔을 어찌할꼬 님은 가버렸구나", 'rd')

['어찌할꼬 가버렸구나']

In [2]:
import pandas as pd

file_path = "../makedata.csv"

df = pd.read_csv(file_path,low_memory=False)

In [3]:
df.head()

,Q,A,id
0,2셋트시키면 4장이 오는 거 맞나요?,2장 구성 상품으로 수량 2개 주문시 총 4장으로 발송되니 참고 부탁드리겠습니다.,64
1,두 장오는 거 맞나요?,두 장 구성 상품이며 옵션 선택 시 색상/색상표기되어 있으니 참고 부탁드리겠습니다.,71
2,요제품 1+1 아닌가요?,해당 쇼핑몰에서 주문해 주시면 1매 증 정품으로 함께 배송되오니 주문 시 참고 부탁...,80
3,지금 구입하면 타이백주나요?,구성 확인 부탁드리겠습니다. 다른 문의사항은 언제든 글 남겨주시구요.,122
4,스냅은 상품당 3개 오는 건가요?,블라인드 상단 브라켓은 사이즈에 맞는 수량으로 배송됩니다 브라켓은 나사로 설치해 주...,280


In [33]:
# 증강된 새로운 행 추가
new_rows = {"Q": [], "A": [], "id": []}
for row in df.itertuples():
    result = generate_augmented_sentences(row[1], 'sr')
    if isinstance(result, list):
        result = result[0]
    new_rows["Q"].append(result)
    new_rows["A"].append(row.A)
    new_rows["id"].append(f"sr-{row.id}")

new_df = pd.DataFrame(new_rows)

In [36]:
new_df[:10]

,Q,A,id
0,셋트시키면 장이 오는 거 맞나요,2장 구성 상품으로 수량 2개 주문시 총 4장으로 발송되니 참고 부탁드리겠습니다.,sr-64
1,두 장오는 거 맞나요,두 장 구성 상품이며 옵션 선택 시 색상/색상표기되어 있으니 참고 부탁드리겠습니다.,sr-71
2,요제품 아닌가요,해당 쇼핑몰에서 주문해 주시면 1매 증 정품으로 함께 배송되오니 주문 시 참고 부탁...,sr-80
3,지금 구입하면 타이백주나요,구성 확인 부탁드리겠습니다. 다른 문의사항은 언제든 글 남겨주시구요.,sr-122
4,스냅은 상품당 개 오는 건가요,블라인드 상단 브라켓은 사이즈에 맞는 수량으로 배송됩니다 브라켓은 나사로 설치해 주...,sr-280
5,빨리 주문해야 담주 안에 설치할 수 있지 않을까요,주문하신 후 주문번호/색상/손잡이 방향/맞춤하실가로 세로 사이즈 남겨주시면 주문건 ...,sr-287
6,진그린 좌우 주문했는데요 색상을 바꾸고 싶어서요 아이보리로 바꿀 수 있나요,주문건 확인 결과 그레이 구매해주신 것으로 확인됩니다 또한 가로 200세로 100일...,sr-307
7,하고 싶은데 선택을 어떻게 해야 되나요,165는 특대형으로 주문하셔야 합니다.,sr-348
8,이면 짜리개가 배송된다는 건가요,네 155가 두 개가 되는 거에요.,sr-368
9,이면 로 가 두 개가 온다는 거 맞는 건가요,주문하신 상품과 같은 상품으로 배송됩니다.,sr-373


In [41]:
df_temp_sr = pd.concat([df,new_df], ignore_index=True)

In [43]:
df_temp_sr[4820:4840]

,Q,A,id
4820,일반형 4개 주문했는데 고급형 3개으로 교환하고 싶은데 비용이 얼마나 필요할까요?,차액은 개당 10800원으로 4개 전부 변경 시총 43200원 비용이 더 발생합니다.,307399
4821,몸 무게도 60밖에 안 나가는데 이건 좀 아닌 거 같은데 AS이거 방석만 따로 보내...,파손된 부품은 신속히 재발송해드리겠습니다.,307665
4822,결제를 두 번 나누어서 진행할 수 있나요?,해당 상품은 두 번으로 결제를 나누어 진행할 수 없습니다.,307806
4823,상품 이 상시 AS는 어떻게 처리되나요. 무상인가요?,상품 불량 시 무료 A/S 및 교환 진행됩니다.,308300
4824,4개 주문하면 사은품 2개 오나여?,2+1 진행 중으로 4개 구매 시 사은품 2개 총 6개 배송됩니다.,308301
4825,선택했는데 국민계좌로 오천원 입금하면 환불되나요?,계좌로 입금 부탁드리겠습니다. 반품 환불 진행하도록 하겠습니다.,308305
4826,4개 구매했으면 2+1이니까 총 6개와야 하는 거 아닌가요?,사은품 추가 출고로 진행도와 드리겠습니다. 문의 남겨주셔서 감사합니다.,308306
4827,와이더 두 개 주문했으면 동일 제품 한 개 추가로 보내주시는 거 맞죠?,주문 내역 확인하였으며 총 3개 배송됩니다.,308313
4828,커블체어와이드 무료 반품되나요?,이벤트 기간 내 주문건으로 무료 반품 가능하며 판매자 송금으로 반품 접수해주시면 됩니다.,308320
4829,반품 배송비 3월 12일 5천원 입금해 드렸는데 반품 배송비 입금 문자가 또 왔네요?,입금 확인되어 반품 완료되었습니다.,308322


In [44]:
df_temp_sr.to_csv("data_sr.csv", index=False, encoding="utf-8-sig")

In [6]:
# 증강된 새로운 행 추가
new_rows = {"Q": [], "A": [], "id": []}
for row in df.itertuples():
    result = generate_augmented_sentences(row[1], 'ri')
    if isinstance(result, list):
        result = result[0]
    new_rows["Q"].append(result)
    new_rows["A"].append(row.A)
    new_rows["id"].append(f"ri-{row.id}")

new_df = pd.DataFrame(new_rows)

In [7]:
new_df[:10]

,Q,A,id
0,셋트시키면 장이 오는 거 맞나요,2장 구성 상품으로 수량 2개 주문시 총 4장으로 발송되니 참고 부탁드리겠습니다.,ri-64
1,두 장오는 거 맞나요,두 장 구성 상품이며 옵션 선택 시 색상/색상표기되어 있으니 참고 부탁드리겠습니다.,ri-71
2,요제품 아닌가요,해당 쇼핑몰에서 주문해 주시면 1매 증 정품으로 함께 배송되오니 주문 시 참고 부탁...,ri-80
3,지금 구입하면 타이백주나요,구성 확인 부탁드리겠습니다. 다른 문의사항은 언제든 글 남겨주시구요.,ri-122
4,개 스냅은 상품당 개 개 오는 건가요,블라인드 상단 브라켓은 사이즈에 맞는 수량으로 배송됩니다 브라켓은 나사로 설치해 주...,ri-280
5,수 수 수 수 빨리 주문해야 담주 안에 설치할 수 있지 않을까요,주문하신 후 주문번호/색상/손잡이 방향/맞춤하실가로 세로 사이즈 남겨주시면 주문건 ...,ri-287
6,진그린 좌우 주문했는데요 색상을 수 바꾸고 싶어서요 수 아이보리로 바꿀 수 수 있나요,주문건 확인 결과 그레이 구매해주신 것으로 확인됩니다 또한 가로 200세로 100일...,ri-307
7,하고 싶은데 선택을 어떻게 해야 되나요,165는 특대형으로 주문하셔야 합니다.,ri-348
8,이면 짜리개가 배송된다는 건가요,네 155가 두 개가 되는 거에요.,ri-368
9,이면 가 가로 가 가 가 두 개가 온다는 거 맞는 건가요,주문하신 상품과 같은 상품으로 배송됩니다.,ri-373


In [8]:
df_temp_ri = pd.concat([df,new_df], ignore_index=True)

In [9]:
df_temp_ri[4820:4840]

,Q,A,id
4820,일반형 4개 주문했는데 고급형 3개으로 교환하고 싶은데 비용이 얼마나 필요할까요?,차액은 개당 10800원으로 4개 전부 변경 시총 43200원 비용이 더 발생합니다.,307399
4821,몸 무게도 60밖에 안 나가는데 이건 좀 아닌 거 같은데 AS이거 방석만 따로 보내...,파손된 부품은 신속히 재발송해드리겠습니다.,307665
4822,결제를 두 번 나누어서 진행할 수 있나요?,해당 상품은 두 번으로 결제를 나누어 진행할 수 없습니다.,307806
4823,상품 이 상시 AS는 어떻게 처리되나요. 무상인가요?,상품 불량 시 무료 A/S 및 교환 진행됩니다.,308300
4824,4개 주문하면 사은품 2개 오나여?,2+1 진행 중으로 4개 구매 시 사은품 2개 총 6개 배송됩니다.,308301
4825,선택했는데 국민계좌로 오천원 입금하면 환불되나요?,계좌로 입금 부탁드리겠습니다. 반품 환불 진행하도록 하겠습니다.,308305
4826,4개 구매했으면 2+1이니까 총 6개와야 하는 거 아닌가요?,사은품 추가 출고로 진행도와 드리겠습니다. 문의 남겨주셔서 감사합니다.,308306
4827,와이더 두 개 주문했으면 동일 제품 한 개 추가로 보내주시는 거 맞죠?,주문 내역 확인하였으며 총 3개 배송됩니다.,308313
4828,커블체어와이드 무료 반품되나요?,이벤트 기간 내 주문건으로 무료 반품 가능하며 판매자 송금으로 반품 접수해주시면 됩니다.,308320
4829,반품 배송비 3월 12일 5천원 입금해 드렸는데 반품 배송비 입금 문자가 또 왔네요?,입금 확인되어 반품 완료되었습니다.,308322


In [10]:
df_temp_ri.to_csv("data_ri.csv", index=False, encoding="utf-8-sig")

In [11]:
# 증강된 새로운 행 추가
new_rows = {"Q": [], "A": [], "id": []}
for row in df.itertuples():
    result = generate_augmented_sentences(row[1], 'rs')
    if isinstance(result, list):
        result = result[0]
    new_rows["Q"].append(result)
    new_rows["A"].append(row.A)
    new_rows["id"].append(f"rs-{row.id}")

new_df = pd.DataFrame(new_rows)

In [12]:
df_temp_rs = pd.concat([df,new_df], ignore_index=True)

In [13]:
df_temp_rs[4820:4840]

,Q,A,id
4820,일반형 4개 주문했는데 고급형 3개으로 교환하고 싶은데 비용이 얼마나 필요할까요?,차액은 개당 10800원으로 4개 전부 변경 시총 43200원 비용이 더 발생합니다.,307399
4821,몸 무게도 60밖에 안 나가는데 이건 좀 아닌 거 같은데 AS이거 방석만 따로 보내...,파손된 부품은 신속히 재발송해드리겠습니다.,307665
4822,결제를 두 번 나누어서 진행할 수 있나요?,해당 상품은 두 번으로 결제를 나누어 진행할 수 없습니다.,307806
4823,상품 이 상시 AS는 어떻게 처리되나요. 무상인가요?,상품 불량 시 무료 A/S 및 교환 진행됩니다.,308300
4824,4개 주문하면 사은품 2개 오나여?,2+1 진행 중으로 4개 구매 시 사은품 2개 총 6개 배송됩니다.,308301
4825,선택했는데 국민계좌로 오천원 입금하면 환불되나요?,계좌로 입금 부탁드리겠습니다. 반품 환불 진행하도록 하겠습니다.,308305
4826,4개 구매했으면 2+1이니까 총 6개와야 하는 거 아닌가요?,사은품 추가 출고로 진행도와 드리겠습니다. 문의 남겨주셔서 감사합니다.,308306
4827,와이더 두 개 주문했으면 동일 제품 한 개 추가로 보내주시는 거 맞죠?,주문 내역 확인하였으며 총 3개 배송됩니다.,308313
4828,커블체어와이드 무료 반품되나요?,이벤트 기간 내 주문건으로 무료 반품 가능하며 판매자 송금으로 반품 접수해주시면 됩니다.,308320
4829,반품 배송비 3월 12일 5천원 입금해 드렸는데 반품 배송비 입금 문자가 또 왔네요?,입금 확인되어 반품 완료되었습니다.,308322


In [14]:
df_temp_rs.to_csv("data_rs.csv", index=False, encoding="utf-8-sig")

In [15]:
# 증강된 새로운 행 추가
new_rows = {"Q": [], "A": [], "id": []}
for row in df.itertuples():
    result = generate_augmented_sentences(row[1], 'rd')
    if isinstance(result, list):
        result = result[0]
    new_rows["Q"].append(result)
    new_rows["A"].append(row.A)
    new_rows["id"].append(f"rd-{row.id}")

new_df = pd.DataFrame(new_rows)

In [16]:
df_temp_rd = pd.concat([df,new_df], ignore_index=True)

In [17]:
df_temp_rd.to_csv("data_rd.csv", index=False, encoding="utf-8-sig")